# Imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn import set_config
set_config(transform_output='pandas')
pd.set_option('display.max_colwidth', 250)

from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential

In [2]:
import joblib
df=joblib.load('Data-NLP/Movies-reviews-processed.joblib')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2419 entries, 1 to 8647
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       2419 non-null   object 
 1   movie_id        2419 non-null   int64  
 2   imdb_id         2419 non-null   object 
 3   original_title  2419 non-null   object 
 4   review          2419 non-null   object 
 5   rating          2419 non-null   float64
 6   group           2419 non-null   object 
 7   cln_review      2419 non-null   object 
 8   tokens          2419 non-null   object 
 9   lemmas          2419 non-null   object 
 10  tokens-joined   2419 non-null   object 
 11  lemmas-joined   2419 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 245.7+ KB


,review_id,movie_id,imdb_id,original_title,review,rating,group,cln_review,tokens,lemmas,tokens-joined,lemmas-joined
1,57086ff5c3a3681d29001512,7443,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy anima...",9.0,high_rating,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy anima...","[guilty, pleasure, personally, love, great, escape, works, seen, years, rightfully, esteemed, british, animation, company, highly, recommended, children, adults, enjoy, animation]","[guilty, pleasure, personally, love, great, escape, works, seen, years, rightfully, esteemed, british, animation, company, highly, recommended, children, adults, enjoy, animation]",guilty pleasure personally love great escape works seen years rightfully esteemed british animation company highly recommended children adults enjoy animation,guilty pleasure personally love great escape works seen years rightfully esteemed british animation company highly recommended children adults enjoy animation
6,57b2d23dc3a36837d7000b14,955,tt0120755,Mission: Impossible II,"The first underwhelmed me, but this one straight-up bored me. Again, of course seeing Hunt climb a mountain without a harness is impressive sure. And I even quite liked the idea behind the villain of the piece (though even that angle was woefully...",3.0,low_rating,"The first underwhelmed me, but this one straight-up bored me. Again, of course seeing Hunt climb a mountain without a harness is impressive sure. And I even quite liked the idea behind the villain of the piece (though even that angle was woefully...","[underwhelmed, straight, bored, course, seeing, hunt, climb, mountain, harness, impressive, sure, liked, idea, villain, piece, angle, woefully, underdeveloped, setting, predominantly, australia, grab, attention, saying, cause, pretty, biased, com...","[underwhelmed, straight, bored, course, seeing, hunt, climb, mountain, harness, impressive, sure, liked, idea, villain, piece, angle, woefully, underdeveloped, setting, predominantly, australia, grab, attention, saying, cause, pretty, biased, com...",underwhelmed straight bored course seeing hunt climb mountain harness impressive sure liked idea villain piece angle woefully underdeveloped setting predominantly australia grab attention saying cause pretty biased comes hear tell mission impossi...,underwhelmed straight bored course seeing hunt climb mountain harness impressive sure liked idea villain piece angle woefully underdeveloped setting predominantly australia grab attention saying cause pretty biased comes hear tell mission impossi...
7,5e9120963679a10018375490,955,tt0120755,Mission: Impossible II,After quite entertainign Mission Impossible I the second installment turned out ... terrible. As if the screenwriters didn't know how to fill the 2 hrs with action the overuse of heroic slow motion scenes is horrible. You almost might need a barf...,2.0,low_rating,After quite entertainign Mission Impossible I the second installment turned out ... terrible. As if the screenwriters didn't know how to fill the 2 hrs with action the overuse of heroic slow motion scenes is horrible. You almost might need a barf...,"[entertainign, mission, impossible, second, installment, turned, terrible, screenwriters, know, fill, 2, hrs, action, overuse, heroic, slow, motion, scenes, horrible, need, barf, bag, stand, slo, mo, seconds, 15, seconds, amazed, decide, push, m,...","[entertainign, mission, impossible, second, installment, turned, terrible, screenwriters, know, fill, 2, hrs, action, overuse, heroic, slow, motion, scenes, horrible, need, barf, bag, stand, slo, mo, seconds, 

In [3]:
df['rating'].value_counts(dropna=False)

9.0     616
10.0    564
4.0     514
1.0     284
3.0     254
2.0     153
9.5      15
0.5      10
3.5       4
1.5       3
2.5       2
Name: rating, dtype: int64

In [6]:
sampler = RandomUnderSampler(random_state=42)
df_ml, _ = sampler.fit_resample(df,df['group'])
df_ml['group'].value_counts()

high_rating    1195
low_rating     1195
Name: group, dtype: int64

In [8]:
X = df_ml['review']
y_string = df_ml['group']

encoder = LabelEncoder()
y = pd.Series(encoder.fit_transform(y_string))
y.value_counts()

0    1195
1    1195
dtype: int64

In [9]:
classes = y.unique()
classes

array([0, 1])

In [14]:
dir(fn)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [10]:
# Adding folder above to python path 
import sys, os
sys.path.append( os.path.abspath("../"))

In [13]:
%load_ext autoreload 
%autoreload 2

import custom_functions as fn